In [1]:
import logging
import importlib
import os, sys
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional, Union, Tuple

sys.path.append(os.environ["HOME"]+"/trading/python/")
from lib import talos_utils
from lib import utils

sys.path.append(os.environ["HOME"]+"/trading/keys/")
import keys

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.10f}'.format

ModuleNotFoundError: No module named 'keys'

In [ ]:
google_sheet_secret = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
dashboard_sheet_name = "Copy of A1 - Trading"


talos_adv_api = 'ANC34AV1RAK2'
talos_adv_secret_api =  'mj9a70zpeve9rnam55kn0ql6c7sze3gw'
talos_host_wl = "tal-160.prod.talostrading.com"

talos_secret_api_sandbox = keys.talos_sandbox_api_secret
talos_sandbox = talos_utils.Talos(talos_api_sandbox, talos_secret_api_sandbox, host_sandbox)

In [ ]:
market_translation = {
    "b2c2/b-2-c-2-a-1" : {"type": "dealer", "full_name" : "B2C2 Overseas, Ltd."},
    "wintermute/wintermute-trading-rfq" : {"type": "dealer", "full_name" : "Wintermute Trading LTD"},
    "okex/okx" : {"type": "exchange", "full_name" : "OKX (Aux Cayes FinTech Co. Ltd.)"},
    "kraken/kraken-a-1" : {"type": "exchange", "full_name" : "Kraken (Payward Trading Ltd.)"},
    "coinbase/coinbase-hrp" : {"type": "exchange", "full_name" : "HRP Coinbase"},
    "bybit/bybit-a-1" : {"type": "exchange", "full_name" : "Bybit (Bybit Fintech Limited)"},
    "galaxy/galaxy-trading-rfq-a-1" : {"type": "dealer", "full_name" : "Galaxy Trading Asia Limited"},
    #sandbox markets
    "coinbase/coinbase" : {"type": "exchange", "full_name" : "Coinbase Hold"},
    "kraken/kraken" : {"type": "exchange", "full_name" : "Kraken Hold"},
    "galaxy/galaxy" : {"type": "dealer", "full_name" : "Galaxy Hold"},
    "jump/jump" : {"type": "dealer", "full_name" : "Jump Hold"},
    "hehmeyer/hehmeyer" : {"type": "dealer", "full_name" : "Hehmeyer Hold"},
    "dvchain/dv-chain" : {"type": "dealer", "full_name" : "DV Chain Hold"},
    "wintermute/wintermute" : {"type": "dealer", "full_name" : "Wintermute Hold"},
}

In [ ]:
talos_wl = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, talos_host_wl)
#not booking automatically yet
#gsheet_key = utils.read_secret(google_sheet_secret)

In [ ]:
def get_order_type(order_response: list) -> str:
    if "-PERP" in order_response[0]["Symbol"] and "MultilegParams" not in order_response[0]:
        return "perp"
    elif "MultilegParams" in order_response[0]:
        return "multileg"
    elif "-PERP" not in order_response[0]["Symbol"]:
        return "spot"
    else:
        return"unknown"

In [ ]:
def construct_orders_entries(order_response: list) -> pd.DataFrame:

    ord_type = get_order_type(order_response)
    order_df = pd.DataFrame(order_response)
    markets_df = pd.DataFrame(order_df["Markets"][0])

    order_id = order_df["OrderID"][0]
    order_denom = order_df["Currency"][0]

    markets_df["timestamp"] = pd.to_datetime(order_df["Timestamp"][0]).strftime("%Y-%m-%d %H:%M:%S")
    markets_df = markets_df[markets_df["CumQty"].astype(float) > 0]
    markets_df["full_name"] = markets_df["MarketAccount"].apply(lambda x: market_translation.get(x, {}).get("full_name", "Market not found"))
    markets_df["type"] = markets_df["MarketAccount"].apply(lambda x: market_translation.get(x, {}).get("type", "Market not found"))
    markets_df["ord_type"] = ord_type
    markets_df["Symbol"] = markets_df["Symbol"].str.replace("-", "/")
    markets_df["Side"] = markets_df["Side"].str.upper()
    markets_df = markets_df.assign(
        FeeCurrency=markets_df.get("FeeCurrency", ""),
        fees="",
        fees_ccy=""
    ).reset_index(drop=True)
    markets_df[['OrderQty', 'CumQty', 'CumAmt', 'AvgPxAllIn', 'CumFee']] = markets_df[['OrderQty', 'CumQty', 'CumAmt', 'AvgPxAllIn', 'CumFee']].apply(pd.to_numeric, errors='coerce')

    if ord_type == "perp":
        markets_df["dealer_trade_id"] = order_id
        buy_mask = markets_df["Side"] == "BUY"
        sell_mask = markets_df["Side"] == "SELL"
        if "Currency" not in markets_df.columns:
            markets_df["Currency"] = order_denom

        markets_df["bought"] = np.where(buy_mask, markets_df["OrderQty"], markets_df["CumAmt"])
        markets_df["bought_asset"] = np.where(buy_mask, markets_df["Currency"], markets_df["AmountCurrency"])
        markets_df["sold"] = np.where(buy_mask, markets_df["CumAmt"], markets_df["OrderQty"])
        markets_df["sold_asset"] = np.where(buy_mask, markets_df["AmountCurrency"], markets_df["Currency"])

        markets_df["fees"] = markets_df["CumFee"]
        markets_df["fees_ccy"] = markets_df["FeeCurrency"]

        markets_df["fee_ccy_same_bought"] = markets_df["fees_ccy"] == markets_df["bought_asset"]
        markets_df["bought"] = np.where((markets_df["fee_ccy_same_bought"]) & (markets_df["Side"] == "SELL"), markets_df["bought"] - markets_df["CumFee"], markets_df["bought"])
        markets_df["sold"] = np.where((~markets_df["fee_ccy_same_bought"]) & (markets_df["Side"] == "BUY"), markets_df["sold"] + markets_df["CumFee"], markets_df["sold"])

        return markets_df[["dealer_trade_id", "timestamp", "full_name", "Side", "Symbol", "bought", "bought_asset", "sold", "sold_asset", "AvgPxAllIn", "fees", "fees_ccy", "CumFee", "FeeCurrency", "OrderQty", "CumQty", "CumAmt", "fee_ccy_same_bought"]]
    
    elif ord_type == "multileg":
        print("multileg")
        markets_df["dealer_trade_id"] = f"{order_id}_" + markets_df["Market"].astype(str) if len(markets_df) > 1 else order_id
        markets_df = markets_df[markets_df["Market"] != "multileg"]
        markets_df.reset_index(drop=True, inplace=True)
        
        if markets_df["Symbol"].str.contains("PERP").any():
            markets_df["contract_size"] = markets_df.query("~Symbol.str.contains('PERP')")["CumQty"].iloc[0] / markets_df.query("Symbol.str.contains('PERP')")["CumQty"].iloc[0]


    elif ord_type == "spot":
        markets_df["dealer_trade_id"] = f"{order_id}_" + markets_df["Market"].astype(str) if len(markets_df) > 1 else order_id
        transacted_df = pd.DataFrame(order_df["TransactedAssets"][0]).set_index("Asset").T
        cols = transacted_df.columns
        transacted_df[cols] = transacted_df[cols].apply(pd.to_numeric, errors='coerce')
        transacted_df = transacted_df.loc[["Amount"]].reset_index(drop=True)

        for col in transacted_df.columns:
            if transacted_df[col].iloc[0] > 0:
                transacted_df["bought_asset"] = [col]
                transacted_df = transacted_df.rename(columns={col: 'bought'})
            elif transacted_df[col].iloc[0] < 0:
                transacted_df["sold_asset"] = [col]
                transacted_df = transacted_df.rename(columns={col: 'sold'})
                transacted_df["sold"] = transacted_df["sold"] * -1
        
        markets_df = pd.concat([markets_df, transacted_df], axis=1).reset_index(drop=True)
        markets_df["AvgPxAllIn"] = np.where((markets_df["AvgPxAllIn"] == 0) & (markets_df["AvgPx"] != 0), markets_df["AvgPx"], markets_df["AvgPxAllIn"])

        if (markets_df["type"] == "exchange").all() and len(markets_df) == 1:
            print("exchange here")
            buy_minus_buy = np.where(markets_df["Side"] == "BUY", markets_df["OrderQty"] - markets_df["bought"], markets_df["CumAmt"] - markets_df["bought"])
            sell_minus_sell = np.where(markets_df["Side"] == "BUY", markets_df["CumAmt"] - markets_df["sold"], markets_df["OrderQty"] - markets_df["sold"])
            
            markets_df["fees"] = np.maximum(buy_minus_buy, sell_minus_sell)
            markets_df["fees_ccy"] = np.where(buy_minus_buy > sell_minus_sell, markets_df["bought_asset"], markets_df["sold_asset"])

            return markets_df[["dealer_trade_id", "timestamp", "full_name", "Side", "Symbol", "bought", "bought_asset", "sold", "sold_asset", "AvgPxAllIn", "fees", "fees_ccy", "CumFee", "FeeCurrency", "OrderQty", "CumQty", "CumAmt"]]


    market_len = len(markets_df)

    for i in range(market_len):
        base_qty = float(markets_df["OrderQty"].iloc[i])
        quote_qty = float(markets_df["CumAmt"].iloc[i])
        base_ccy = markets_df["Currency"].iloc[i]
        quote_ccy = markets_df["AmountCurrency"].iloc[i]
        side = markets_df["Side"].iloc[i]
        avg_px_allin = float(markets_df["AvgPxAllIn"].iloc[i])
        talos_reported_fee = markets_df["CumFee"].iloc[i]
        talos_reported_fee_ccy = markets_df["FeeCurrency"].iloc[i]

        if side == "BUY":
            markets_df.loc[i, "bought"] = base_qty
            markets_df.loc[i, "bought_asset"] = base_ccy
            markets_df.loc[i, "sold"] = quote_qty
            markets_df.loc[i, "sold_asset"] = quote_ccy

        elif side == "SELL":
            markets_df.loc[i, "bought"] = quote_qty
            markets_df.loc[i, "bought_asset"] = quote_ccy
            markets_df.loc[i, "sold"] = base_qty
            markets_df.loc[i, "sold_asset"] = base_ccy


        markets_df.loc[i,"fees"] = talos_reported_fee
        markets_df.loc[i,"fees_ccy"] = talos_reported_fee_ccy
        
    if markets_df["type"].str.contains("dealer").all():
        markets_df["matching_ctpy_id"] = ""
        return markets_df[["dealer_trade_id", "matching_ctpy_id", "timestamp", "full_name", "Side", "Symbol", "bought", "bought_asset", "sold", "sold_asset", "AvgPxAllIn", "fees", "fees_ccy", "CumFee", "FeeCurrency", "OrderQty", "CumQty", "CumAmt"]]

    return markets_df[["dealer_trade_id", "timestamp", "full_name", "Side", "Symbol", "bought", "bought_asset", "sold", "sold_asset", "AvgPxAllIn", "fees", "fees_ccy", "CumFee", "FeeCurrency", "OrderQty", "CumQty", "CumAmt"]]

In [ ]:
### If you want to test some order ids:

perp_order_id_sell = "0b4cdeef-66d4-4a56-8032-275357d41cd1"
perp_order_id_buy = "8a036d9b-3afa-45c1-ac86-53801b8bd58e"
spot_order_id_dealer = "8d1c417c-be83-4296-b73c-a8d42e425bad"
spot_order_id_exchange_buy = "36f36134-6650-4c3c-a024-4a4f794dbf05"
spot_order_id_exchange_sell = "9f9c0496-3c8b-4631-88d3-66d0eeee672d"
multileg_delta_one_order_id = "509832f-55db-4db9-813f-2239640f7c1c"
multileg_synthetic_order_id = "7509832f-55db-4db9-813f-2239640f7c1c"
kraken_pyusd_no_fees_bug = "14111373-8427-44f6-a622-25929f542129"

In [ ]:

order_id = multileg_synthetic_order_id

test_order = talos_wl.get_orders(order_id=order_id)
construct_orders_entries(test_order).iloc[:,:12]

multileg


,dealer_trade_id,timestamp,full_name,Side,Symbol,bought,bought_asset,sold,sold_asset,AvgPxAllIn,fees,fees_ccy
0,7509832f-55db-4db9-813f-2239640f7c1c_kraken,2024-11-17 19:02:19,Kraken (Payward Trading Ltd.),BUY,XRP/USD,"17,934.6331587100",XRP,"18,913.8813181100",USD,1.0555667100,17.3204065200,USD
1,7509832f-55db-4db9-813f-2239640f7c1c_wintermute,2024-11-17 19:02:19,Wintermute Trading LTD,SELL,XRP/USD,"18,932.2200000000",USD,"17,934.6340000000",XRP,1.0556281300,0.0000000000,NaN


In [2]:
import pandas as pd
### DEALER DEALER DUMP

# List of multileg_synthetic_order_ids
order_ids = data = [
    "da76271e-b48b-4040-8965-63c5b17bc1cd",
    "2dfa2ce2-6c8c-490c-9efb-ed550fcdd83d",
    "6518baf9-3d67-4047-be01-19a08322593f",
    "88f6c627-92cb-495e-a5a0-0e4df354196d",
    "de6e943d-aa7b-4911-8ee9-2206a57ae919",
    "b6daf03d-89e9-4dfe-9a83-ee3746b36bd4",
    "e07e6990-4def-49fb-a35d-f5920cd03537",
    "42e35f03-d70e-4fa7-820b-bc06a914de88",
    "8cd96b54-a0f6-44a3-9d93-ee96104ff3d6",
    "08654402-1e2f-42f0-9d50-a0545f8310e4",
    "58626fbf-f2cd-4f72-ac46-f7b84e0a1e64",
    "108c673e-6e5b-4f69-86c6-bbb6d83859da",
    "94d0c7fa-926c-404e-acff-da18ac64c0c4",
    "95e33f8c-ae7e-45eb-ba06-681d0bf1c8fc",
    "82e4946c-5b21-4bc9-834f-5ece302809d3",
    "e76c5fc3-89a0-49aa-9676-c33a3337cb33",
    "dbd4ac40-c9c4-45da-bc6f-7567de072ecf"
]












# You can now use uuid_list for further operations
print(order_ids)











# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each order ID
for order_id in order_ids:
    # Fetch the test order using the given order_id
    test_order = talos_wl.get_orders(order_id=order_id)
    
    # Construct the order entries and select the first 12 columns
    order_entries = construct_orders_entries(test_order).iloc[:, :12]
    
    # Append the DataFrame to the list
    dataframes.append(order_entries)

# Concatenate all DataFrames into a single DataFrame
final_dataframe = pd.concat(dataframes, ignore_index=True)

# Display the final DataFrame
display(final_dataframe)


['da76271e-b48b-4040-8965-63c5b17bc1cd', '2dfa2ce2-6c8c-490c-9efb-ed550fcdd83d', '6518baf9-3d67-4047-be01-19a08322593f', '88f6c627-92cb-495e-a5a0-0e4df354196d', 'de6e943d-aa7b-4911-8ee9-2206a57ae919', 'b6daf03d-89e9-4dfe-9a83-ee3746b36bd4', 'e07e6990-4def-49fb-a35d-f5920cd03537', '42e35f03-d70e-4fa7-820b-bc06a914de88', '8cd96b54-a0f6-44a3-9d93-ee96104ff3d6', '08654402-1e2f-42f0-9d50-a0545f8310e4', '58626fbf-f2cd-4f72-ac46-f7b84e0a1e64', '108c673e-6e5b-4f69-86c6-bbb6d83859da', '94d0c7fa-926c-404e-acff-da18ac64c0c4', '95e33f8c-ae7e-45eb-ba06-681d0bf1c8fc', '82e4946c-5b21-4bc9-834f-5ece302809d3', 'e76c5fc3-89a0-49aa-9676-c33a3337cb33', 'dbd4ac40-c9c4-45da-bc6f-7567de072ecf']


NameError: name 'talos_wl' is not defined

In [14]:
import pandas as pd
### BYBIT DEALER DUMP

# List of multileg_synthetic_order_ids
order_ids = ids = [
    "f76dce14-75a6-473e-970c-fe7f170bb542"
]


# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each order ID
for order_id in order_ids:
    # Fetch the test order using the given order_id
    test_order = talos_wl.get_orders(order_id=order_id)
    
    # Construct the order entries and select the first 12 columns
    order_entries = construct_orders_entries(test_order).iloc[:, :12]
    
    # Append the DataFrame to the list
    dataframes.append(order_entries)

# Concatenate all DataFrames into a single DataFrame
final_dataframe = pd.concat(dataframes, ignore_index=True)

# Display the final DataFrame
display(final_dataframe)


multileg


,dealer_trade_id,timestamp,full_name,Side,Symbol,bought,bought_asset,sold,sold_asset,AvgPxAllIn,fees,fees_ccy
0,f76dce14-75a6-473e-970c-fe7f170bb542_wintermute,2025-03-09 12:10:01,Wintermute Trading LTD,SELL,BTC/USDT,"2,000.5880000000",USDT,0.0236799700,BTC,"84,484.4127456100",0.0000000000,NaN
1,f76dce14-75a6-473e-970c-fe7f170bb542_bybit,2025-03-09 12:10:01,Bybit (Bybit Fintech Limited),BUY,BTC/USDT,0.0236800000,BTC,"2,000.9262943800",USDT,"84,515.0369103200",0.3897796564,USDT
